In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "C:\\Users\\gpath\\Downloads\\Aviation_KPIs_Dataset.xlsx"
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name="Sheet1")

# Outlier detection using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Removing outliers from Profit (USD)
df_cleaned = detect_outliers_iqr(df, "Profit (USD)")

# Feature Engineering: Extracting time-based features
df_cleaned["Departure Hour"] = df_cleaned["Scheduled Departure Time"].dt.hour
df_cleaned["Departure Month"] = df_cleaned["Scheduled Departure Time"].dt.month
df_cleaned["Departure Day"] = df_cleaned["Scheduled Departure Time"].dt.dayofweek

# Drop unnecessary columns
df_ml = df_cleaned.drop(columns=["Flight Number", "Scheduled Departure Time", "Actual Departure Time"])

# Define features (X) and target variable (y)
X = df_ml.drop(columns=["Profit (USD)"])
y = df_ml["Profit (USD)"]

# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# EDA: Correlation Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_cleaned.corr(numeric_only=True), annot=False, cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

# Profit Distribution
plt.figure(figsize=(10, 5))
sns.histplot(df_cleaned["Profit (USD)"], bins=50, kde=True, color="blue")
plt.title("Profit Distribution")
plt.xlabel("Profit (USD)")
plt.ylabel("Frequency")
plt.show()

# Scatter Plots for Key Features
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.scatterplot(x=df_cleaned["Delay (Minutes)"], y=df_cleaned["Profit (USD)"], ax=axes[0], alpha=0.5)
axes[0].set_title("Profit vs Delay")

sns.scatterplot(x=df_cleaned["Load Factor (%)"], y=df_cleaned["Profit (USD)"], ax=axes[1], alpha=0.5, color="green")
axes[1].set_title("Profit vs Load Factor")

sns.scatterplot(x=df_cleaned["Revenue (USD)"], y=df_cleaned["Profit (USD)"], ax=axes[2], alpha=0.5, color="red")
axes[2].set_title("Profit vs Revenue")

plt.tight_layout()
plt.show()

# Print dataset shapes
print("Training data shape:", X_train_scaled.shape, y_train.shape)
print("Testing data shape:", X_test_scaled.shape, y_test.shape)

Exploratory Data Analysis (EDA) Insights

Correlation Heatmap
Strong correlation between Revenue and Profit (expected).
Load Factor (%) has a positive impact on Profit.
Operating Cost and Profit are negatively correlated.

Profit Distribution
Most flights have profits clustered around $5,000 - $20,000.
Some flights are operating at a loss.

Feature-Target Relationships
Higher Delays → Lower Profit (flights with long delays tend to be less profitable).
Higher Load Factor → Higher Profit (better seat occupancy improves revenue).
Higher Revenue → Higher Profit, but not always (operating costs play a role).